In [3]:
# | default_exp benchmark.classification

In [4]:
from ts.benchmark.tsdataset import TimeSeriesBenchmarkDataset

In [5]:
benchmark = TimeSeriesBenchmarkDataset()
ds_list = benchmark.task_datasets["classification"]
df = benchmark.load_dataset("EthanolConcentration")
df = df.iloc[:500]

In [11]:
from ts.classification.cnnclassifer import TimeSeriesDataModule, TSNDTensorClassifier
from ts.tsfeatures.ts2image import transform_ts2img_tensor

In [12]:
df.head()

NameError: name 'df' is not defined

In [13]:
# transform_ts2img_tensor(df, data_dir='EthanolConcentration_classification', categorical_label=True, label_col='label')

In [14]:
import json

with open("EthanolConcentration_classification/classes.json", "r") as file:
    classes = json.load(file)

In [15]:
import torch

x = torch.load("EthanolConcentration_classification/0.pt")["image"]

In [16]:
x.shape

torch.Size([9, 1751, 1751])

In [17]:
ds = TimeSeriesDataModule(
    data_dir="EthanolConcentration_classification", batch_size=4, num_workers=12
)

model = TSNDTensorClassifier(
    model_name="efficientnet_b0",
    num_classes=len(classes),
    in_channels=x.shape[0],
    reduced_channels=3,
    input_size=x.shape[-1],
    output_size=min(255, x.shape[-1]),
)

In [18]:
import wandb
from pytorch_lightning.loggers import WandbLogger

wandb_logger = WandbLogger(
    project="ts-classification", name="cnn.model=efficientnet_b0.ds=model_classifier"
)
wandb_logger.experiment.config["model"] = "efficientnet_b0"
wandb_logger.experiment.config["ds"] = "model_classifier"
wandb_logger.experiment.config["finetune"] = False

In [19]:
import pytorch_lightning as pl

trainer = pl.Trainer(
    logger=wandb_logger,
    accelerator="auto",
    devices=[0],
    min_epochs=1,
    max_epochs=100,
    enable_checkpointing=True,
    callbacks=[
        pl.callbacks.EarlyStopping("val_loss", patience=5, verbose=False),
    ],
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [20]:
ckpt_path = "model_checkpoints/cnn_model_classification.ckpt"
finetune = False
if finetune:
    trainer.fit(model, ds, ckpt_path=ckpt_path)
else:
    trainer.fit(model, ds)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                        | Params | Mode 
-------------------------------------------------------------------------
0 | preprocessor     | ChannelReducerAndDownscaler | 2.2 K  | train
1 | pretrained_model | EfficientNet                | 4.3 M  | train
2 | criterion        | CrossEntropyLoss            | 0      | train
3 | accuracy         | MulticlassAccuracy          | 0      | train
4 | f1_score         | MulticlassF1Score           | 0      | train
5 | auc              | MulticlassAUROC             | 0      | train
-------------------------------------------------------------------------
331 K     Trainable params
4.0 M     Non-trainable params
4.3 M     Total params
17.355    Total estimated model params size (MB)
352       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Validation: |                                                                                                 …


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
trainer.save_checkpoint("model_checkpoints/cnn_model_classification.ckpt")
trainer.test(model, ds);